<a href="https://colab.research.google.com/github/kartoch/colab-eda/blob/master/99_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projet Gas (2018-2019)

## Présentation

### Sujet

Le but de ce projet est d'étudier un jeu de données en commençant par les étapes ETL (*Extract-Transform-Load*), l'exploration des données puis l'étude d'une ou plusieurs questions ouvertes.

### Dataset

Les fichiers de données utilisés dans le cadre de ce projet sont ceux des tarifs des pompes à essence en France.

Ils sont disponibles ici: https://www.prix-carburants.gouv.fr/rubrique/opendata/ 

Durant le TP en salle, le mieux est de faire une copie locale des fichiers sur mon compte dans le répertoire temporaire de la machine (/tmp).

In [0]:
!mkdir /tmp/gas

In [0]:
!cp ~jcartign/datasets/PrixCarburants_*.zip /tmp/gas

### Logiciels étudiés

* Numpy et Pandas
* Seaborn et matplotlib

Si vous avez fait votre installation avec anaconda ou miniconda, voici la commande pour installer ces logiciels:

In [0]:
%conda install pytables fastparquet numpy seaborn matplotlib pandas

### Organisation du projet

Le projet doit être organisé en une série de feuilles Jupyter numérotées.

Il y a (au moins) 4 feuilles à rendre:

* conversion des XML en CSV
* chargement des CSV dans pandas, typage des données et conversion dans le format parquet
* exploration des données
* étude d'une (ou plusieurs) questions ouvertes

### Organisation des feuilles

La règle de base est que l’ensemble de vos feuilles soit **réutilisable**, c’est-à-dire qu’une personne peut reprendre un projet et relancer l’exécution de chacune (et éventuellement aboutir au  même résultat, mais le sujet de reproductibilité des expérimentations n’est pas couvert dans ce projet).

Afin d'aider sur l'ordre d'exécution de vos feuilles, nommer les en commençant avec un numéro pour savoir dans quelle ordre elles doivent êtres exécutées, par exemple 01-csv-generation, 02-data-typing, etc.

### Organisation d'une feuille

Vos feuilles doivent êtres structurés comment suit
1. La première cellule doit être en markdown, avec un titre et un ou plusieurs paragraphes décrivant le but de la feuille.
2. La seconde cellule doit contenir les constantes qu’une personne utilisant votre feuille peut-être intéressée de modifier (chemin vers des datasets par exemple).

In [0]:
INPUT_PATH_TO_XML_FILES="/home/kartoch/Documents/datasets/gas/"
OUTPUT_PATH_TO_CSV_FILES="/tmp/gas"

3. La troisième cellule doit contenir l’ensemble des imports de la feuille suivi éventuellement de l’initialisation de certain composants (par exemple définition de la taille des graphiques générés par matplotlib ou seaborn).

In [0]:
import csv
import io
import xml.sax
import zipfile

## Structuration du code

Quelques remarques sur comment structurer votre code
* Ne pas avoir plusieurs lignes par cellule (les résultats des lignes intermediaires ne sont pas affichés)
* Éviter dans la mesure du possible les commentaires dans votre code python, préférer une cellule markdown au dessus de votre code décrivant le comportement et/ou l'algorithme impliqué dans la cellule. Vous pouvez aussi ajouter de l'information pour un méthode sous forme de docstring

In [0]:
def method_qui_fait_quelque_chose():
    """Cette méthode fait quelque chose"""
    pass

* Utiliser des *backslash* pour éviter que votre code déborde sur plusieurs lignes

In [0]:
"bonjour les amis"\
     .split(" ")

['bonjour', 'les', 'amis']

* Les chaînes de caractères peuvent être répartis sur plusieurs lignes

In [0]:
a = "abc" +\
    "def"

* les paramètres peuvent êtres répartis sur plusieurs lignes

In [0]:
"blabla".replace("bla",
                 "BLA")

'BLABLA'

## **Première feuille:** Conversion des XML en CSV

La première feuille doit lire l'ensemble des XML représentant les données étudiés de manière hierarchique et sauvegarder ces données avec le format CSV (le format le plus commun pour des *datasets*). Il est possible de décompresser les fichiers zip à la volée ou de le faire manuellement avant l'exécution de la feuille.

### Lecture des fichiers zip

Soit un zip accédé par l'object fichier retourné par `io.BytesIo` (un module simulant les accès à un fichier binaire representé par une chaîne de caractère), voici comment ouvrir ce fichier zip et lire le contenu du fichier à l'intérieur:

In [0]:
f = io.BytesIO(b'\x50\x4b\x03\x04\x0a\x00\x00\x00\x00\x00\x8f\x61\x84\x4e\x6b\xeb\x27\x37\x07\x00'
               b'\x00\x00\x07\x00\x00\x00\x0a\x00\x1c\x00\x63\x6f\x75\x63\x6f\x75\x2e\x74\x78\x74'
               b'\x55\x54\x09\x00\x03\x8d\xd8\xa5\x5c\x92\xd8\xa5\x5c\x75\x78\x0b\x00\x01\x04\xe9'
               b'\x03\x00\x00\x04\x64\x00\x00\x00\x63\x6f\x75\x63\x6f\x75\x0a\x50\x4b\x01\x02\x1e'
               b'\x03\x0a\x00\x00\x00\x00\x00\x8f\x61\x84\x4e\x6b\xeb\x27\x37\x07\x00\x00\x00\x07'
               b'\x00\x00\x00\x0a\x00\x18\x00\x00\x00\x00\x00\x01\x00\x00\x00\xa4\x81\x00\x00\x00'
               b'\x00\x63\x6f\x75\x63\x6f\x75\x2e\x74\x78\x74\x55\x54\x05\x00\x03\x8d\xd8\xa5\x5c'
               b'\x75\x78\x0b\x00\x01\x04\xe9\x03\x00\x00\x04\x64\x00\x00\x00\x50\x4b\x05\x06\x00'
               b'\x00\x00\x00\x01\x00\x01\x00\x50\x00\x00\x00\x4b\x00\x00\x00\x00\x00')

zip_ref = zipfile.ZipFile(f, 'r')
[inside_filename] = zip_ref.namelist()
print("inside_filename: " + inside_filename)
inside_data = zip_ref.open(inside_filename).readline()
print("inside_data: " + str(inside_data))
zip_ref.close()

inside_filename: coucou.txt
inside_data: b'coucou\n'


## Lecture des fichiers zippés à partir de Google Storage

In [0]:
from io import BytesIO
import requests
from zipfile import ZipFile

In [0]:
# Start year
START_YEAR=2007

# End year
END_YEAR=2007

BASE_URL = "https://storage.googleapis.com/essence-dataset-eda/PrixCarburants_annuel_"

PATH_TO_CSV_FILES = "/tmp/"

In [0]:
def download_datasets(year):
        url = BASE_URL + str(year) + ".zip"
        r = requests.get(url)
        return ZipFile(BytesIO(r.content))
        
def generator_zip_file():
    for year in range(START_YEAR,END_YEAR+1):
        zip_ref = download_datasets(year)        
        [xml_filename] = zip_ref.namelist()
        yield (zip_ref.open(xml_filename),year)
        zip_ref.close()

In [4]:
for file,year in generator_zip_file():
    print("file is the file descriptor",file)
    print("year",year)

file is the file descriptor <zipfile.ZipExtFile name='PrixCarburants_annuel_2007.xml' mode='r' compress_type=deflate>
year 2007


### Lecture des fichiers XML

Il existe plusieurs méthodes pour lire un xml:
1. xml.etre.ElementTrree: mécanisme simple représentant l'intégralité de la structure XML en mémoire
2. xml.sax: une implémentation de SAX2 permettant de définir les actions à prendre lors du traîtement d'un fichier XML
3. lxml: non couvert dans ce cours

Dans le cadre de ce projet, le premier n'est pas utilisable au vu de la taille des données chargées. L'approche SAX2 est décrite ci-dessous.

Le principe de SAX2 est de lire le fichier tout en appelant à chaque entité une fonction fournie par le programmeur.

Soit par exemple le fichier suivant (émulé grace au module `io.StringIO` simulant l'accès à un fichier texte): 

In [0]:
my_xml = io.StringIO('<abc id="toto">'
                       '<cool fih="123" koka="kola">trop</cool>'
                       '<def>méthodes</def>'
                     '</abc>')

Le programmeur implémente les méthodes nécessaires dans la classe suivante:

In [0]:
class StreamHandler(xml.sax.handler.ContentHandler):

    def startElement(self, name, attrs):
        """Appelé à chaque tag ouvrant, par exemple <coucou>. Les attributs du tag sont 
           disponibles sous forme de dictionnaire.
           """
        print("startElement " + name + " attrs:" + str(dict(attrs.items())))

    def characters(self, content):
        """Appelé à chaque texte entre les tags, par exemple pour 
           <coucou>ICI</coucou>AUSSI<tralala/> la méthode sera appelé 
           pour ICI puis pour AUSSI
           """
        print("characters " + content)
        
    def endElement(self, name):
        """Appelé à chaque tag fermant, par exemple </coucou>"""
        print("endElement " + name)

In [0]:
parser = xml.sax.make_parser()
handler = StreamHandler()
parser.setContentHandler(handler)
parser.parse(my_xml)

startElement abc attrs:{'id': 'toto'}
startElement cool attrs:{'fih': '123', 'koka': 'kola'}
characters trop
endElement cool
startElement def attrs:{}
characters méthodes
endElement def
endElement abc


**Attention:** selon l'encodage des caractères, la méthode `characters` peut-être appelé plusieurs fois sur une seule chaîne.

### Écriture des CSV

Voici un exemple d'utilisation du module CSV disponible dans la librarie standard de Python.

In [0]:
csv_file = io.StringIO()
csv_writer = csv.DictWriter(csv_file, fieldnames=['a','b'])
csv_writer.writeheader()
csv_writer.writerow({'a': "blabla", 'b': "tagada"})
print(csv_file.getvalue())

a,b
blabla,tagada



## **Seconde feuille:** Typage des données

La seconde feuille commence par ouvrir les fichiers CSV avec pandas. Le sujet de pandas ayant été couvert auparavant, nous ne nous interesserons qu'au problème de conversion des données temporelles.

## **Troisième feuille:** Découverte graphique des données

### Rendu graphique

Il existe plusieurs bibliothèque pour l'exploration des données. Nous utiliserons ici seaborn et pyplot.

**TODO**

### Questions à répondre
* Première exploration de la tendance des prix: Moyenne des prix par rapport au temps pour chaque carburant (nécessité d'utiliser [le mécansisme de sliding window de pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html) pour affiner les courbes)
* Nombre de MAJ par mois (voir l'usage de [Grouper](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Grouper.html) comme option de groupBy)
* Fréquence des mise-à-jour par proportion de stations 
* Proportion de chaque services (avec un barplot)
* Est ce que les informations sur les PDV évoluent chaque année ?
* Variation du nombre de PDV // année
* Scatterplot du prix en fonction du temps pour chaque carburant

## **Feuilles projets** 

Voici une liste de proposition de sujets ouverts à couvrir pendant le projet EDA

## **Premier sujet:** Prix et station essence

### Classification des pompes en fonction de leur prix moyen

Est il possible de classer chaque pompe dans une catégorie en fonction de son prix moyen (le nombre de catégorie étant fixé, par exemple 5) ?

Cette question en pose plusieurs autres:

* est ce que la catégorisation d'une pompe à essence est indépendante du type d'essence ?
* quel est l'impact des erreurs (voir autres sujets ci-dessous) ?

### Affichage graphique des prix 

Il est alors possible d'afficher les graphiques du prix moyen de chaque catégorie de pompe en fonction de son carburant.

### Prix VS autoroutes

A partir du résultat précédent, est il possible de coréeler le fait qu'une station est sur l'autoroute (code de 'pop') et son prix ?

### Représentation géographique

Créer un graphique pour identifier les variation de prix en fonction de la position géographique:
* un point par station avec un code couleur pour chaque catégorie
* une heat map avec une couleur basé sur la moyenne des catégories de la zone couverte

## **Second sujet:** Qualité des données de prix

### Erreurs sur les relevés des prix

Certaines stations envoient des prix eronnés. Il est possible de procéder de plusieurs manières (complémentaires) pour les enlever:
1. détecter les prix abberants (à partir de valeurs hors normes par exemple)
2. vérifier le comportement local (variation importante d'une même courbe)
3. vérifier le comportement global (différence par rapport aux autres prix dans la même période)

### Catégoriser les stations en fonction de leur taux d'erreur

Est il possible de donner un indice de confiance pour chaque pompe sur la qualité de ses prix avec plusieurs méthodes (combinables):
1. à partir de son taux de rafraichissement ?
2. ses erreurs sur les prix (voir précédent sujet) ?
3. de son comportement par rapport aux autres pompes ?